# Computer Components MOO Problem

The objective of this exercise is to select computer components in order to build a computer satisfying the following objective:

$$Minimize\ [C_t(x),P_t(x)]; \ Maximize \ Q_t(x) \ subject \ to \ C_t(x) \leq 1700; P_t(x) \leq 2.5; r_i > 2.5$$

Where:

$$ \begin{array}{lcl} C_t(x) & =  & \sum c_i * x_i \\ 
Q_t(x) & = & \sum (w_i *r_i) * x_i \\
P_t(x) & = & \sum p_i * x_i \end{array} $$

Being:
- c_i: cost of a component, price
- w_i: weight associated to the component type, for calculation purposes, out of 1. See table bellow
- r_i: review index, quality
- p_i: weight of the component (kg), weight
- x_i: mask, 1 if the component is selected, 0 otherwise 

|Component | w_i   |
|----------|-------|
|  cpu     | 0.3   |
|  disk    | 0.2   |
|  board   | 0.1   |
|  ram     | 0.3   |
|  gpu     | 0.1   |
|  Total   |   1   |

## Dataset loading and filtering

In [1]:
import pandas as pd

df = pd.read_csv('./delivery/pieces.csv')
print(df.head())
print(df.shape)

  type       price  quality    weight
0  cpu  172.158304      3.5  0.258227
1  cpu  120.820045      3.0  0.229969
2  cpu  131.290487      4.5  0.117316
3  cpu  110.145139      3.5  0.204912
4  cpu  123.840845      4.5  0.235732
(1000, 4)


One of the requirements of the problem is that:
$$ r_i > 2.5 $$
For this, elements not satisfying this condition will be removed from the dataset

In [2]:
df.drop(df[df.quality <= 2.5].index, inplace=True)
print(df.shape)
print(df.describe())

(792, 4)
            price     quality      weight
count  792.000000  792.000000  792.000000
mean   230.121710    3.751263    0.249275
std    128.266797    0.567781    0.086155
min     17.091812    3.000000    0.100609
25%    136.895647    3.000000    0.174929
50%    180.004637    4.000000    0.245327
75%    295.520759    4.500000    0.324295
max    671.883945    4.500000    0.399811


## Problem definition and solving

First the objective functions used in the problem are defined:

In [3]:
def obtain_selected_vars_no(selected_vars):
    ''' This function returns the total
        selected vars number '''
    return sum([1 if selected_vars[i] else 0 for i in range(len(selected_vars))])

In [4]:
def calculate_total_weight(selected_vars, weights_array):
    ''' This function returns the total weight'''
    return sum([weights_array[i] if selected_vars[i] else 0 for i in range(len(selected_vars))])

In [5]:
def calculate_total_cost(selected_vars, costs_array):
    ''' This function returns the total cost '''
    return sum([costs_array[i] if selected_vars[i] else 0 for i in range(len(selected_vars))])

In [6]:
def calculate_total_quality(selected_vars, quality_array):
    ''' This function returns the total quality '''
    return sum([quality_array[i] if selected_vars[i] else 0 for i in range(len(selected_vars))])

Then, the function handling the problem's logic is defined:

In [7]:
from platypus import NSGAII, Problem, Constraint, Binary, nondominated, unique
import functools
    
def computer_components_problem(x, arg1, arg2, arg3):
    ''' This function defines the logic of the 
        computer components selection problem '''
    selection = x[0]
    no_items_selected = obtain_selected_vars_no(selection)
    total_quality = calculate_total_quality(selection, arg3)
    total_weight = calculate_total_weight(selection, arg1)
    total_cost = calculate_total_cost(selection, arg2)
    
    
    return [total_quality,total_weight,total_cost] , [total_weight, no_items_selected, total_cost]

In this step the data is obtained:

In [10]:
items = 7
max_weight = 9
max_cost = 15
no_items = 2
weights = [2, 3, 6, 7, 5, 9, 4]
cost = [6, 5, 8, 9, 6, 7, 3]
quality = [3, 5, 4, 2, 1, 1, 5]

Now, the problem is defined:

In [11]:
problem = Problem(1, 3, 3)
problem.types[0] = Binary(items)
problem.directions[0] = Problem.MAXIMIZE
problem.directions[1:] = Problem.MINIMIZE
problem.constraints[0] = Constraint("<=", max_weight)
problem.constraints[1] = Constraint("<=", no_items)
problem.constraints[2] = Constraint("<=", max_cost)
problem.function = functools.partial(computer_components_problem, arg1=weights, arg2=cost, arg3 = quality)

In [12]:
algorithm = NSGAII(problem)
algorithm.run(10000)

for solution in unique(nondominated(algorithm.result)):
    print(solution.variables, solution.objectives)

[[False, False, False, False, False, False, False]] [0, 0, 0]
[[False, True, False, False, False, False, True]] [10, 7, 8]
[[True, True, False, False, False, False, False]] [8, 5, 11]
[[True, False, False, False, False, False, False]] [3, 2, 6]
[[False, False, False, False, False, False, True]] [5, 4, 3]
[[True, False, False, False, False, False, True]] [8, 6, 9]
[[False, True, False, False, False, False, False]] [5, 3, 5]
